In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import pathpy as pp
import networkx as nx 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import itertools as it
import seaborn as sns
import pickle
import matplotlib
import os
import imageio
import time
import datetime
import statistics
from natsort import natsorted
import sklearn as sk
from sklearn.linear_model import LinearRegression as LinReg
import dateutil.parser as dp
import matplotlib.patches as patches

In [2]:
poss = None
medie = list()
gweights = list()
midpoint = 0

In [3]:
def map_id_names(x):
    if x == 6980735383087397986:
        return "LG"
    if x == 196762576653771958:
        return "Francesco"
    if x == 7019585581295147828:
        return "Francesca"
    if x == 8809629864047484994:
        return "Antonio"

In [4]:
def get_dict(df):
    tmp = dict()
    res = True
    s_min = df.rssi.min()
    s_max = df.rssi.max()
    s_avg = df.rssi.mean()
    divider = "$%&"
    for i, r in df.iterrows():
        scanner = str(r['scanner'])
        advertiser = str(r['advertiser'])
        tmp[scanner+divider+advertiser] = list() 
    for index, row in df.iterrows():
        s = str(row['scanner'])
        a = str(row['advertiser'])
        #esiste già
        if s+divider+a in tmp:
            l = tmp[s+divider+a]
            l.append(int(row['rssi']))
            tmp[s+divider+a]=l 
    return tmp

In [5]:
def get_g_nodes(pos, df):
    print("x")

In [6]:
def get_1m_subdataset(df):
    df['time'] =  pd.to_datetime(conn['time'],utc=True)
    df['type'] = np.where(conn.time.dt.time < datetime.time(10, 26, 10), 'ok', 'no')
    df = conn.sort_values(by="time")
    sub = df.loc[df['type'] == "ok"]
    return sub.rssi.median()

In [7]:
def get_pos(df):
    G = nx.Graph()
    divider = "$%&"
    tmp = get_dict(df)
    #s_min = df.rssi.min()
    #s_max = df.rssi.max()
    s_avg = df.rssi.mean()
    for key in tmp:
        k = key.split(divider)
        s1 = k[0]
        s2 = k[1]
        l1 = list()
        l2 = list()
        x1 = False
        x2 = False
        
        if s1+divider+s2 in tmp:
            l1 = tmp[s1+divider+s2]
            x1 = True
        if s2+divider+s1 in tmp:
            l2 = tmp[s2+divider+s1]
            x2 = True
        

        l3 = l1 + l2

        if len(l3)!=0:
            m = statistics.mean(l3)
            #questo lo devo fare solo se m è minore di una certa soglia
            G.add_edge(s1, s2, weight=round(m, 2))
            res = True
        
        if x1:
            tmp[s1+divider+s2] = list()
        if x2:
            tmp[s2+divider+s1] = list()
    
    pos=nx.spring_layout(G,seed=60) # pos = nx.nx_agraph.graphviz_layout(G)zz
    
    return pos

In [8]:
def fakeRSSI(df, qqq, nodes):
    G = nx.Graph()
    #t = pp.TemporalNetwork()
    divider = "$%&"
    tmp = get_dict(df)
    s_avg = df.rssi.mean()
    for node in nodes:
        G.add_node(node)

    for key in tmp:
        k = key.split(divider)
        s1 = k[0]
        s2 = k[1]
        l1 = list()
        l2 = list()
        x1 = False
        x2 = False
        
        if s1+divider+s2 in tmp:
            l1 = tmp[s1+divider+s2]
            x1 = True
        if s2+divider+s1 in tmp:
            l2 = tmp[s2+divider+s1]
            x2 = True
        

        l3 = l1 + l2

        if len(l3)!=0:
            m = statistics.mean(l3)
            #questo lo devo fare solo se m è minore di una certa soglia
            #if m < s_avg:
            global medie
            medie.append(round(m, 2))
            G.add_edge(s1, s2, weight=round(m, 2))
            res = True
        
        if x1:
            tmp[s1+divider+s2] = list()
        if x2:
            tmp[s2+divider+s1] = list()

In [28]:
def creategraphRSSI(df, qqq, nodes):
    G = nx.Graph()
    #t = pp.TemporalNetwork()
    global midpoint
    divider = "$%&"
    tmp = get_dict(df)
    s_avg = df.rssi.mean()
    for node in nodes:
        G.add_node(node)

    for key in tmp:
        k = key.split(divider)
        s1 = k[0]
        s2 = k[1]
        l1 = list()
        l2 = list()
        x1 = False
        x2 = False
        
        if s1+divider+s2 in tmp:
            l1 = tmp[s1+divider+s2]
            x1 = True
        if s2+divider+s1 in tmp:
            l2 = tmp[s2+divider+s1]
            x2 = True
        

        l3 = l1 + l2

        if len(l3)!=0:
            m = statistics.mean(l3)
            #questo lo devo fare solo se m è minore di una certa soglia
            G.add_edge(s1, s2, weight=round(m, 2))
            res = True
        
        if x1:
            tmp[s1+divider+s2] = list()
        if x2:
            tmp[s2+divider+s1] = list()
    
    
     # pos = nx.nx_agraph.graphviz_layout(G)
    global poss        
    widths = nx.get_edge_attributes(G, 'weight')
    nodelist = G.nodes()
    x = list(widths.values())
    if len(x) == 0:
        return False
    m = max(x)
   
    
    #plt.text(left, bottom, 't='+str(index), color='black', 
    #    bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'),
    #    horizontalalignment='left',
    #    verticalalignment='bottom',)
    
    fig = plt.figure(figsize=(13, 10))
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    labels = nx.get_edge_attributes(G,'weight')
    
    blues = plt.cm.Greens
    reds = plt.cm.Reds
    
    #x = [w for u, v, w in G.edges(data=True)]
    #print(x)
    h = [blues(w['weight']) if w['weight'] > midpoint else reds(w['weight']) for u, v, w in G.edges(data=True)]
    
    nx.draw(G, poss, node_size=1500, node_color='lightgrey', edgelist=edges, edge_color=h,
            width=list(map(lambda number: number * 16 / m, x)))
    nx.draw_networkx_edge_labels(G,poss,edge_labels=labels)
    nx.draw_networkx_labels(G, pos=poss)
    
    
    
    v = f'{qqq}'
    plt.title("t="+v +" [s]", fontsize=30,  x=0.45, y=0.95)
   
    fig.savefig('img/plot' + v + ".png")
    #plt.show()
    plt.close(fig=None)
    return res

In [11]:
def nonefunct(elem):
    return elem

In [12]:
np.set_printoptions(suppress=True)
conn = pd.read_csv("connections_norm_test1.csv",index_col=False)
global midpoint
midpoint = get_1m_subdataset(conn)
conn['time'] =  pd.to_datetime(conn['time'],utc=True)
conn['type'] = np.where(conn.time.dt.time > datetime.time(10, 29, 0), 'collo', 'tasca')
tasca, collo = [x for _, x in conn.groupby(conn['type'] == "collo")]
df1 = pd.DataFrame(tasca)
df2 = pd.DataFrame(collo)
df1['scanner'] = df1['scanner'].apply(lambda x: map_id_names(x))
df1['advertiser'] = df1['advertiser'].apply(lambda x: map_id_names(x))

dfz = list()
az = df1.groupby(['scanner','advertiser'])
for group_key, group_value in az:
    group = az.get_group(group_key)
    dfz.append(group)
    
#magic numbers    
l_min = 10000
l_max = -10000

for elem in dfz:
    l = len(elem)
    if l < l_min:
        l_min = l
    if l > l_max:
        l_max = l

df1 = df1.sort_values(by="time").reset_index(drop=True)
df1.head()

,Unnamed: 0,time,scanner,advertiser,power,rssi,rssi_norm,type
0,0,2022-02-23 10:24:15+00:00,LG,Francesco,-7,-76,-76.000000,tasca
1,1,2022-02-23 10:24:15+00:00,LG,Francesco,-7,-76,-76.000000,tasca
2,2,2022-02-23 10:24:15+00:00,LG,Francesco,-7,-74,-74.000000,tasca
3,12,2022-02-23 10:24:16+00:00,Francesco,LG,-7,-74,-79.073584,tasca
4,10,2022-02-23 10:24:16+00:00,Francesco,LG,-7,-68,-75.137959,tasca


In [13]:
#prendo il numero di nodi presenti
uniques = set()
scan = df1.scanner.unique()
adv = df1.advertiser.unique()

for e in scan:
    uniques.add(e)
for a in adv:
    uniques.add(a)

nodes = list(uniques)
print(nodes)

['Francesco', 'LG', 'Francesca', 'Antonio']


In [14]:
subdatasets = list()
resampled = df1.set_index('time').resample('1s')
for i, (timestamp,df) in enumerate(resampled):
    subdatasets.append(df)

In [26]:
#iniziamo a graficare
k = 0
x = get_pos(subdatasets[70])
global poss
poss = x
for fake in subdatasets:
    fakeRSSI(fake, k, nodes)



for elem in subdatasets:
    res = creategraphRSSI(elem, k, nodes)
    if res:
        k+=1

(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(0.9686274509803922, 0.9882352941176471, 0.9607843137254902, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(0.9686274509803922

(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.9411764705882353, 1.0)
(1.0, 0.9607843137254902, 0.941

In [27]:
#creiamo il video
png_dir = 'img/'
images = []
for file_name in natsorted(os.listdir(png_dir)):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave('movie.gif', images, fps=4)
print("finished")

finished
